## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [14]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 66.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 144.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 215.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 184.6 MB/s eta 0:00:00


In [49]:
# For installing the libraries & downloading models from HF Hub
!python3 -m venv rag-env
!source rag-env/bin/activate

!pip install --upgrade pip
!pip install numpy==1.24.4 scipy==1.10.1 sentence-transformers langchain chromadb

Error: Command '['/content/rag-env/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: rag-env/bin/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Succ

In [1]:
# Core libraries
import os
import json
#import pandas as pd
import tiktoken

# LangChain components
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Hugging Face and LLM
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [2]:
## Model configuration
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#### Response

In [3]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
# function to generate, process, and return the response from the LLM
def generate_llama_response(user_prompt):

    # System message
    system_message = """
    [INST]<<SYS>> Respond to the user question based on the user prompt<</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_llama_response(user_prompt)
print(response)

 The management of sepsis in a critical care unit (CCU) follows a standardized protocol that includes early recognition, aggressive fluid resuscitation, antibiotic therapy, and close monitoring. Here are the key components of the protocol:

1. Early Recognition: The CCU team should be vigilant in identifying patients at risk for sepsis, using criteria such as SIRS (Systemic Inflammatory Response Syndrome) or QSOFA (Quick Sequential Organ Failure Assessment).
2. Aggressive Fluid Resuscitation: Patients with sepsis should receive intravenous fluids and vasopressors as needed to maintain mean arterial pressure ≥65 mmHg and urine output ≥0.5 mL/kg/min.
3. Antibiotic Therapy: Broad-spectrum antibiotics should be administered promptly, based on the suspected etiology of sepsis, and guided by blood cultures and other diagnostic tests. The choice of antibiotics may vary depending on the severity of sepsis and the patient's medical history and allergies.
4. Close Monitoring: Patients with sepsi

#### Observation
The response provides a clear, structured overview of sepsis management in critical care, aligning well with standard protocols like early fluids, antibiotics, and monitoring. It’s comprehensive and clinically relevant, though it could benefit from including fluid types, lactate trending, and source control measures.


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response = generate_llama_response(user_prompt)
print(response)

Llama.generate: prefix-match hit


 Sure, I'd be happy to help! Appendicitis is a medical emergency that requires prompt treatment. Here are some common symptoms of appendicitis:

1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen. The pain may be constant or it may come and go.
2. Nausea and vomiting.
3. Loss of appetite.
4. Fever (usually above 101°F).
5. Abdominal tenderness and guarding (muscle tension) when the abdomen is touched.
6. Abdominal swelling or bloating.
7. Diarrhea or constipation.
8. Inability to pass gas.
9. A feeling of urgency to have a bowel movement.
10. Weakness and fatigue.

If you suspect that you or someone else may have appendicitis, it is important to seek medical attention immediately. Appendicitis can be treated with antibiotics, but surgery is usually necessary to remove the inflamed appendix. The surgical procedure for treating appendicitis is called an appendectomy.

There are two types of appendectomies: open and

#### Observation
The response gives a detailed and accurate summary of appendicitis symptoms and treatment. It correctly emphasizes that surgery (usually an appendectomy) is the standard treatment, while antibiotics may be used in select cases. The explanation of both laparoscopic and open procedures is clear and clinically appropriate.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response = generate_llama_response(user_prompt)
print(response)

Llama.generate: prefix-match hit


 Sudden patchy hair loss, also known as alopecia areata, can be a distressing condition that affects millions of people worldwide. While there is no definitive cure for this condition, there are several effective treatments and solutions that can help promote hair growth and reduce the size of bald spots.

     One of the most common treatment options for alopecia areata is topical corticosteroids, which can be applied directly to the affected area to suppress the immune system and promote hair growth. Another effective treatment option is anthralin, a medication that is applied to the skin to reduce inflammation and stimulate hair growth.

     In addition to these medical treatments, there are also several natural remedies that can help promote hair growth and reduce the size of bald spots. These include coconut oil, which contains antioxidants and fatty acids that nourish the scalp and promote healthy hair growth; rosemary essential oil, which has anti-inflammatory properties that c

#### Observation
The response provides a clear overview of alopecia areata, outlining both medical and natural treatment options effectively. It highlights the importance of identifying underlying causes and consulting a healthcare provider, making the guidance both informative and responsible.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response = generate_llama_response(user_prompt)
print(response)

Llama.generate: prefix-match hit


 Treatments for a physical injury to brain tissue depend on the severity and location of the injury, as well as the specific symptoms and needs of the individual. Here are some common treatments that may be recommended:

1. Medications: Various medications may be prescribed to manage symptoms such as pain, inflammation, seizures, or mood changes. These can include analgesics, anti-inflammatory drugs, anticonvulsants, and mood stabilizers.
2. Rehabilitation therapy: Physical, occupational, and speech therapy may be necessary to help the individual regain lost function and improve their quality of life. These therapies can include exercises to strengthen affected muscles, activities to improve cognitive and motor skills, and training in the use of assistive devices such as walkers or wheelchairs.
3. Surgery: In some cases, surgery may be necessary to relieve pressure on brain tissue, repair damaged blood vessels, or remove a tumor or other foreign object that is causing injury.
4. Stem c

#### Observation
The response offers a well-rounded overview of treatment options for brain injuries, from medications and surgery to advanced therapies like stem cells and hyperbaric oxygen. It effectively emphasizes individualized care and multidisciplinary rehabilitation, reflecting a realistic and informed medical approach.


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response = generate_llama_response(user_prompt)
print(response)

Llama.generate: prefix-match hit


 Sure, I'd be happy to help! If someone has fractured their leg during a hiking trip, it is essential to take immediate action to provide proper care and prevent further complications. Here are some necessary precautions and treatment steps that should be considered:

1. Stop activity: The first step is to stop any physical activity and rest the affected limb immediately. This will help prevent further damage or displacement of the fracture.
2. Assess the injury: Check for any signs of swelling, deformity, or open wounds near the site of the fracture. If there are any of these symptoms, call for emergency medical assistance immediately.
3. Immobilize the leg: Use a splint or brace to immobilize the affected leg and prevent movement. This will help reduce pain and promote healing.
4. Elevate the leg: Elevate the injured leg above the level of your heart to reduce swelling and improve blood flow.
5. Apply ice: Apply ice packs or cold compresses to the affected area for 15-20 minutes at a

#### Observation

The response provides a comprehensive and practical guide for managing a leg fracture during a hiking trip. It covers first aid, pain management, and emergency considerations clearly and effectively. Overall, it's well-structured, medically sound, and appropriate for both laypersons and first responders.


## Question Answering using LLM with Prompt Engineering

In [ ]:
#Generates and compares multiple LLM responses to a medical question using different prompt tuning parameters. .
#Useful for evaluating how decoding settings impact answer quality.

def build_prompt(user_question):
    return f"""### Instruction:
You are a medical expert. Answer the question clearly and concisely.

### Question:
{user_question}

### Answer:"""

# Different parameter combinations
param_combos = [
    {"temperature": 0.3, "top_p": 0.95, "max_tokens": 512, "top_k": 50, "repeat_penalty": 1.1},
    {"temperature": 0.7, "top_p": 0.9,  "max_tokens": 512, "top_k": 40, "repeat_penalty": 1.0},
    {"temperature": 0.1, "top_p": 1.0,  "max_tokens": 1024, "top_k": 100, "repeat_penalty": 1.2},
    {"temperature": 0.5, "top_p": 0.85, "max_tokens": 768, "top_k": 60, "repeat_penalty": 1.15},
    {"temperature": 0.9, "top_p": 0.95, "max_tokens": 512, "top_k": 80, "repeat_penalty": 1.0},
]

# Function to run responses for a dynamic question
def generate_tuned_responses(user_question):
    all_responses = []

    for i, params in enumerate(param_combos):
        prompt = build_prompt(user_question)
        print(f"\n\n=== Response {i+1} === [Params: {params}]")

        output = lcpp_llm(
            prompt=prompt,
            temperature=params["temperature"],
            top_p=params["top_p"],
            max_tokens=params["max_tokens"],
            top_k=params["top_k"],
            repeat_penalty=params["repeat_penalty"],
            echo=False,
        )

        response_text = output["choices"][0]["text"].strip()
        all_responses.append((params, response_text))
        print(response_text)

    return all_responses

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
question = "What is the protocol for managing sepsis in a critical care unit?"
generate_tuned_responses(question)



=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


The management of sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and supportive care, close monitoring, and timely escalation to more intensive therapies when necessary. Here are the key components of the protocol for managing sepsis in a critical care unit:

1. Early recognition: Sepsis is a medical emergency that requires prompt recognition and treatment. The clinical presentation of sepsis can vary, but common signs include fever, tachycardia, tachypnea, confusion, and altered mentation. All patients with suspected sepsis should be immediately evaluated and monitored closely.
2. Administration of antibiotics: Broad-spectrum antibiotics should be administered as soon as possible, ideally within the first hour of recognition of sepsis. The choice of antibiotic should be guided by the suspected source of infection and the patient's allergies and medical history.
3. Supportive care: Patients with

Llama.generate: prefix-match hit


The protocol for managing sepsis in a critical care unit typically involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics, close monitoring of vital signs and laboratory values, and aggressive fluid resuscitation. Here are the key steps in the protocol:

1. Early recognition: Sepsis is a life-threatening condition that requires prompt recognition and treatment. Critical care nurses and physicians should be vigilant in monitoring patients for signs of sepsis, such as fever, tachycardia, tachypnea, and altered mental status.
2. Administration of antibiotics: Broad-spectrum antibiotics should be administered as soon as possible, preferably within the first hour of recognition of sepsis. The choice of antibiotics should be guided by the suspected source of infection and the patient's allergies and sensitivities.
3. Close monitoring of vital signs and laboratory values: Patients with sepsis should be closely monitored for changes in vital

...

Llama.generate: prefix-match hit


The management of sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and supportive care, close monitoring, and timely escalation to more intensive therapies as needed. The specific protocol for managing sepsis may vary depending on the institution, but generally includes the following steps:

1. Early recognition: Identify patients at risk for sepsis using clinical criteria such as fever, tachycardia, tachypnea, and signs of organ dysfunction. Use a validated scoring system like SEPSIS-1 or qSOFA to assess the severity of sepsis.
2. Administer broad-spectrum antibiotics: Start administering antibiotics within the first hour of recognition of sepsis, and continue until there is resolution of signs of infection. The choice of antibiotic should cover common causes of sepsis such as Gram-positive and Gram-negative bacteria, including Pseudomonas aeruginosa.
3. Supportive care: Provide mechanical ventil

Llama.generate: prefix-match hit


The protocol for managing sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt treatment, and careful monitoring of patients with sepsis. Here are the key components of the protocol:

1. Early recognition: Sepsis is a medical emergency, and early recognition is crucial to preventing mortality. The protocol should include a systematic approach to identifying patients at risk of sepsis, such as those with suspected or confirmed infections, organ dysfunction, or signs of severe sepsis.
2. Initial assessment: The protocol should outline the initial assessment of patients with suspected sepsis, including blood cultures, complete blood count, serum lactate, and electrolyte levels. This information is essential for identifying the site and severity of infection, assessing the patient's physiological status, and guiding initial therapy.
3. Rapid administration of antibiotics: The protocol should emphasize the prompt administration of broa

[({'temperature': 0.3,
   'top_p': 0.95,
   'max_tokens': 512,
   'top_k': 50,
   'repeat_penalty': 1.1},
  "The management of sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and supportive care, close monitoring, and timely escalation to more intensive therapies when necessary. Here are the key components of the protocol for managing sepsis in a critical care unit:\n\n1. Early recognition: Sepsis is a medical emergency that requires prompt recognition and treatment. The clinical presentation of sepsis can vary, but common signs include fever, tachycardia, tachypnea, confusion, and altered mentation. All patients with suspected sepsis should be immediately evaluated and monitored closely.\n2. Administration of antibiotics: Broad-spectrum antibiotics should be administered as soon as possible, ideally within the first hour of recognition of sepsis. The choice of antibiotic should be guided by the 

#### Observation
Prompt Design is Effective:
Using ### Instruction, ### Question, and ### Answer clearly structured the LLM's output, keeping responses relevant and concise.

Parameter Variations Affect Detail & Style:
Lower temperatures (0.1–0.3) gave more clinical, focused responses. Higher temperatures (0.7–0.9) added more descriptive or conversational content.

Core Content Consistency:
Across all parameter combinations, key elements of sepsis management (e.g., early recognition, fluid resuscitation, antibiotic use) remained consistent, indicating robustness of the base prompt.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
question = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_tuned_responses(question)



=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


Appendicitis is a medical emergency that occurs when the appendix, a small pouch-like organ attached to the large intestine, becomes inflamed and infected. The common symptoms of appendicitis include:

* Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen
* Nausea and vomiting
* Loss of appetite
* Fever
* Abdominal tenderness and guarding (muscle tension)
* Abdominal swelling

In some cases, the symptoms may be milder and may resemble those of a less severe condition such as a stomach flu or food poisoning. However, if left untreated, appendicitis can lead to serious complications, including the rupture of the appendix and potentially life-threatening infections.

While antibiotics may be used to treat mild cases of appendicitis, surgery is usually necessary to remove the inflamed appendix. The surgical procedure most commonly used to treat appendicitis is a laparoscopic appendectomy, where a small incision is made i

Llama.generate: prefix-match hit


Common symptoms of appendicitis include:

* Sudden and severe pain in the abdomen, usually in the lower right area
* Nausea and vomiting
* Loss of appetite and fever
* Abdominal tenderness and guarding (muscle tension)
* Abdominal swelling and palpable mass

If the appendicitis is caught early, it may be treated with antibiotics and fluids to manage the symptoms and help the inflammation subside. However, if the appendix has already ruptured, surgery is necessary to remove the inflamed appendix and prevent further complications.

The surgical procedure to treat appendicitis is an appendectomy, which can be performed through a laparoscopic or open incision. In a laparoscopic appendectomy, several small incisions are made in the abdomen, and a laparoscope (a thin, lighted tube with a camera) is inserted to visualize the appendix. The inflamed appendix is then removed through one of the small incisions.

In an open appendectomy, a single larger incision is made in the abdomen to allow the

...

Llama.generate: prefix-match hit


Common Symptoms of Appendicitis:

* Sudden severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen
* Nausea and vomiting
* Loss of appetite
* Fever
* Abdominal tenderness and guarding (muscle tension)
* Abdominal swelling
* Diarrhea or constipation

Can Appendicitis be Cured via Medicine?

No, appendicitis cannot be cured solely with medicine. Antibiotics may be prescribed to treat any secondary infections, but the only effective treatment for appendicitis is surgical removal of the inflamed appendix.

Surgical Procedure for Treating Appendicitis:

The most common surgical procedure for treating appendicitis is a laparoscopic appendectomy, where a small incision is made in the abdomen and a laparoscope (a thin tube with a camera and light) is inserted to visualize the inflamed appendix. The surgeon then removes the inflamed appendix through the small incision.

In some cases, an open appendectomy may be necessary if the 

Llama.generate: prefix-match hit


Common symptoms for appendicitis include:

1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.
2. Nausea and vomiting.
3. Fever and chills.
4. Loss of appetite.
5. Abdominal tenderness and guarding (muscle tension).

If the appendicitis is caught early, it can be treated with antibiotics and fluid management. However, if the appendix has already ruptured, surgery is necessary to remove the inflamed appendix.

The surgical procedure for treating appendicitis is usually an appendectomy, which involves removing the inflamed appendix through an incision in the abdomen. The type of appendectomy performed may depend on the severity of the appendicitis and the patient's overall health. The two types of appendectomies are:

1. Open appendectomy: This is the traditional method, where a single incision is made in the lower right abdomen to remove the inflamed appendix.
2. Laparoscopic appendectomy: This is a minimally inva

[({'temperature': 0.3,
   'top_p': 0.95,
   'max_tokens': 512,
   'top_k': 50,
   'repeat_penalty': 1.1},
  'Appendicitis is a medical emergency that occurs when the appendix, a small pouch-like organ attached to the large intestine, becomes inflamed and infected. The common symptoms of appendicitis include:\n\n* Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen\n* Nausea and vomiting\n* Loss of appetite\n* Fever\n* Abdominal tenderness and guarding (muscle tension)\n* Abdominal swelling\n\nIn some cases, the symptoms may be milder and may resemble those of a less severe condition such as a stomach flu or food poisoning. However, if left untreated, appendicitis can lead to serious complications, including the rupture of the appendix and potentially life-threatening infections.\n\nWhile antibiotics may be used to treat mild cases of appendicitis, surgery is usually necessary to remove the inflamed appendix. The surg

#### Observation

The responses consistently identify key appendicitis symptoms such as abdominal pain, nausea, vomiting, fever, and tenderness. All responses agree that while antibiotics may help in mild or early cases, surgery (typically laparoscopic or open appendectomy) is the definitive treatment. There's slight variation in wording and detail due to different temperature and decoding settings. Higher temperatures (like 0.9) show more varied phrasing, while lower ones (like 0.1 or 0.3) provide more factual, consistent responses. Overall, the outputs are medically accurate and align with clinical understanding.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
question = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_tuned_responses(question)



=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


Sudden patchy hair loss, also known as alopecia areata, can be caused by a variety of factors, including autoimmune disorders, fungal infections, and stress. Effective treatments for this condition include:

1. Corticosteroid injections or topical creams: These medications can help suppress the immune system and promote hair growth.
2. Minoxidil (Rogaine): This over-the-counter solution can help stimulate hair growth and slow down hair loss.
3. Anthralin (Psoralen): This medication can help reduce inflammation and promote hair growth.
4. Phototherapy: Exposure to ultraviolet light can help reduce inflammation and promote hair growth.
5. Wigs or hairpieces: These can be used to cover up bald spots and boost confidence.
6. Platelet-rich plasma (PRP) therapy: This is a non-surgical treatment that involves injecting platelet-rich plasma into the scalp to stimulate hair growth.
7. Low-level laser therapy (LLLT): This is a non-invasive treatment that uses low-level lasers or light-emitting d

Llama.generate: prefix-match hit


Sudden patchy hair loss, also known as alopecia areata, can be caused by a variety of factors, including autoimmune disorders, fungal infections, and stress. Effective treatments for this condition include:

1. Topical corticosteroids: These medications are applied directly to the affected area to reduce inflammation and promote hair growth.
2. Minoxidil: This medication is applied to the scalp to stimulate hair growth and slow down hair loss.
3. Anthralin: This medication is applied to the scalp to reduce inflammation and promote hair growth.
4. Phototherapy: Exposure to specific wavelengths of light can help to reduce inflammation and promote hair growth.
5. Immunotherapy: This treatment uses injections or oral medications to suppress the immune system and prevent it from attacking hair follicles.

Possible causes of sudden patchy hair loss include:

1. Autoimmune disorders: In these conditions, the immune system mistakenly attacks healthy tissues, including hair follicles.
2. Fungal

...

Llama.generate: prefix-match hit


Sudden patchy hair loss, also known as alopecia areata, can have various underlying causes. Here are some effective treatments and solutions for addressing this condition:

1. Corticosteroid injections or topical creams: These medications can help suppress the immune system's attack on hair follicles, promoting hair growth and reducing inflammation.
2. Minoxidil (Rogaine): This over-the-counter solution stimulates hair growth by increasing blood flow to the scalp and widening hair follicles.
3. Anthralin: A medicated cream or ointment that can help suppress the immune system's attack on hair follicles, promoting hair growth.
4. Phototherapy with UVB light: Exposure to specific wavelengths of ultraviolet light can help reduce inflammation and promote hair growth.
5. Platelet-rich plasma (PRP) therapy: Injecting platelet-rich plasma into the affected area can stimulate hair growth by promoting cellular regeneration and healing.
6. Low-level laser therapy (LLLT): This non-invasive treatme

Llama.generate: prefix-match hit


Sudden patchy hair loss, commonly seen as localized bald spots on the scalp, can be caused by a variety of factors. Here are some effective treatments and solutions:

1. Medications: Minoxidil (Rogaine) and finasteride (Propecia) are two medications that are FDA-approved for treating hair loss. Minoxidil can help stimulate hair growth and slow down hair loss, while finasteride works by blocking the production of dihydrotestosterone (DHT), a hormone that contributes to hair loss.
2. Low-level laser therapy (LLLT): LLLT uses low-level lasers or light-emitting diodes to stimulate hair growth. It is thought that the light energy increases blood flow to the scalp, promoting hair growth.
3. Platelet-rich plasma (PRP) therapy: PRP therapy involves injecting platelet-rich plasma (PRP) into the scalp. PRP is derived from the patient's own blood and contains growth factors that can stimulate hair growth.
4. Diet and lifestyle changes: Making changes to your diet and lifestyle can help promote ha

[({'temperature': 0.3,
   'top_p': 0.95,
   'max_tokens': 512,
   'top_k': 50,
   'repeat_penalty': 1.1},
  "Sudden patchy hair loss, also known as alopecia areata, can be caused by a variety of factors, including autoimmune disorders, fungal infections, and stress. Effective treatments for this condition include:\n\n1. Corticosteroid injections or topical creams: These medications can help suppress the immune system and promote hair growth.\n2. Minoxidil (Rogaine): This over-the-counter solution can help stimulate hair growth and slow down hair loss.\n3. Anthralin (Psoralen): This medication can help reduce inflammation and promote hair growth.\n4. Phototherapy: Exposure to ultraviolet light can help reduce inflammation and promote hair growth.\n5. Wigs or hairpieces: These can be used to cover up bald spots and boost confidence.\n6. Platelet-rich plasma (PRP) therapy: This is a non-surgical treatment that involves injecting platelet-rich plasma into the scalp to stimulate hair growth

#### Observation

All five responses identify core appendicitis symptoms and agree that surgical removal is the primary treatment. Lower temperature responses (like 0.1 and 0.3) are more precise and fact-based, while higher ones (0.7 and 0.9) introduce more variation and conversational tone. Despite parameter differences, all responses maintain clinical accuracy.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
question = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_tuned_responses(question)



=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


The treatment for a person with a physical injury to brain tissue depends on the severity and location of the injury, as well as the extent of any impairment of brain function. In general, the goals of treatment are to:

1. Stabilize the patient and prevent further injury or damage to the brain.
2. Manage any acute symptoms such as bleeding, swelling, or seizures.
3. Promote healing and recovery of damaged brain tissue.
4. Address any long-term cognitive, emotional, or behavioral changes that may result from the injury.

Some common treatments for physical injuries to brain tissue include:

1. Medications: To control seizures, reduce swelling, and manage pain.
2. Surgery: To relieve pressure on the brain, repair damaged blood vessels, or remove a tumor or other foreign object that may be causing the injury.
3. Rehabilitation therapy: To help the patient regain lost functions and improve cognitive, emotional, and behavioral abilities. This may include physical therapy, occupational ther

Llama.generate: prefix-match hit


Treatment for a physical injury to brain tissue depends on the severity and location of the injury, as well as the specific symptoms and impairments experienced by the individual. In general, the goal of treatment is to minimize further damage, promote healing, and improve functional abilities.

For temporary impairment of brain function, treatment may include:

1. Rest and avoidance of exacerbating factors: Avoid activities that may exacerbate the injury, such as heavy lifting, bending, or stress.
2. Medications: Pain relievers, anti-inflammatory drugs, and sedatives may be prescribed to manage symptoms and promote rest.
3. Physical therapy: Gentle exercises and stretches may be recommended to maintain range of motion and prevent muscle atrophy.
4. Cognitive therapy: Techniques such as memory aids, problem-solving strategies, and emotional support may be helpful in managing cognitive and behavioral changes.
5. Rehabilitation: A multidisciplinary team of healthcare professionals, inclu

...

Llama.generate: prefix-match hit


Treatment options for a person with a physical injury to brain tissue depend on the severity and location of the injury. In general, the goals of treatment are to relieve symptoms, improve cognitive and physical function, and enhance quality of life. Recommended treatments may include:

1. Medications: To manage symptoms such as pain, seizures, or mood changes.
2. Surgery: To repair the damaged brain tissue, relieve pressure, or remove a blood clot.
3. Rehabilitation therapy: To improve cognitive and physical function, including speech, language, memory, and motor skills.
4. Lifestyle modifications: To address changes in daily living activities, such as using assistive devices or modifying the home environment.
5. Cognitive training: To improve attention, memory, problem-solving, and other cognitive functions.
6. Behavioral therapy: To manage behavioral changes, such as depression, anxiety, or personality changes.
7. Psychosocial support: To address emotional and social challenges, inc

Llama.generate: prefix-match hit


The treatment for a brain injury depends on the severity and location of the injury, as well as the specific symptoms and impairments experienced by the individual. In general, the goals of treatment are to:

1. Stabilize the patient and manage any acute symptoms, such as bleeding or swelling in the brain.
2. Address any underlying medical conditions, such as seizures or infections.
3. Improve cognitive, physical, and emotional functioning through rehabilitation therapies.
4. Provide support and resources for the patient and their family.

Some common treatments for brain injuries include:

1. Medications: to manage symptoms such as pain, seizures, or agitation.
2. Surgery: to relieve pressure on the brain, repair damaged tissue, or remove a blood clot.
3. Rehabilitation therapies: such as physical, occupational, and speech therapy to help the patient regain lost function and improve their quality of life.
4. Cognitive and behavioral therapies: to help the patient manage cognitive and 

[({'temperature': 0.3,
   'top_p': 0.95,
   'max_tokens': 512,
   'top_k': 50,
   'repeat_penalty': 1.1},
  "The treatment for a person with a physical injury to brain tissue depends on the severity and location of the injury, as well as the extent of any impairment of brain function. In general, the goals of treatment are to:\n\n1. Stabilize the patient and prevent further injury or damage to the brain.\n2. Manage any acute symptoms such as bleeding, swelling, or seizures.\n3. Promote healing and recovery of damaged brain tissue.\n4. Address any long-term cognitive, emotional, or behavioral changes that may result from the injury.\n\nSome common treatments for physical injuries to brain tissue include:\n\n1. Medications: To control seizures, reduce swelling, and manage pain.\n2. Surgery: To relieve pressure on the brain, repair damaged blood vessels, or remove a tumor or other foreign object that may be causing the injury.\n3. Rehabilitation therapy: To help the patient regain lost fu

#### Observation

All five responses outline common treatments for brain tissue injury—such as medications, surgery, rehabilitation, and cognitive therapies. Lower-temperature responses (e.g., 0.1 and 0.3) are more structured and factual, with consistent core treatments. Higher-temperature outputs (like 0.7 and 0.9) introduce more variation and detail, including emotional support, assistive tech, or alternative therapies. Response 3 is the most comprehensive, while Response 5 is more conversational and human-focused. Overall, all emphasize the importance of individualized, multidisciplinary care.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
question = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_tuned_responses(question)



=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


If someone has fractured their leg while hiking, it is essential to seek medical attention as soon as possible. The following precautions and treatment steps can be taken:

1. Stop any bleeding: Apply direct pressure to the wound to stop any bleeding. Elevate the affected limb above heart level to reduce swelling.
2. Immobilize the leg: Use a splint or brace to immobilize the leg and prevent further movement. This will help reduce pain and prevent further damage.
3. Assess for neurovascular injury: Check for numbness, tingling, or loss of sensation in the affected limb. Also, assess for pulse and temperature changes.
4. Manage pain: Administer pain medication as prescribed by a healthcare professional.
5. Monitor for infection: Keep the wound clean and dry, and monitor for signs of infection such as redness, swelling, or increased pain.
6. Consider surgery: Depending on the severity of the fracture, surgery may be necessary to realign and stabilize the bones.
7. Follow-up care: After t

Llama.generate: prefix-match hit


If a person has fractured their leg during a hiking trip, there are several necessary precautions and treatment steps that should be taken to ensure proper care and recovery.

1. Stabilize the leg: The first priority is to stabilize the leg to prevent further injury. This can be done using a splint or a cast to immobilize the affected leg.
2. Assess for other injuries: It's important to assess the person for other potential injuries, such as head or spinal injuries, and provide appropriate treatment.
3. Call for help: If the person is in a remote area, call for help as soon as possible. If they are in a more accessible area, call for an ambulance or other emergency services.
4. Keep the leg elevated: Elevating the leg above the level of the heart can help reduce swelling and improve blood flow.
5. Manage pain: Pain management is crucial to ensure the person is comfortable and can rest. This can be achieved using pain medication, ice packs, or other methods as recommended by a medical p

...

Llama.generate: prefix-match hit


If someone has fractured their leg while hiking, it is essential to seek medical attention as soon as possible. In the meantime, there are several precautions and treatment steps that can be taken to ensure proper care and recovery.

Precautions:

1. Immobilize the affected limb: Use a splint or brace to immobilize the fractured leg to prevent further injury or displacement of the bone.
2. Control bleeding: Apply direct pressure to any wounds to control bleeding and minimize swelling.
3. Monitor vital signs: Check the person's pulse, breathing rate, and blood pressure regularly.
4. Keep them warm: Use a blanket or other covering to keep the person warm and prevent hypothermia.
5. Elevate the affected limb: Elevate the fractured leg above the level of the heart to reduce swelling and pain.

Treatment steps:

1. X-rays or imaging tests: Once medical attention is received, the person will need to undergo X-rays or other imaging tests to confirm the diagnosis and determine the extent of th

Llama.generate: prefix-match hit


If a person has fractured their leg during a hiking trip, it is essential to seek medical attention as soon as possible. In the meantime, there are several necessary precautions and treatment steps that can be taken to ensure proper care and recovery.

1. Immobilization: The affected leg should be immobilized using a splint or cast to prevent further damage and promote healing.
2. Pain management: Pain medication should be administered to manage pain and discomfort.
3. Elevation: The affected leg should be elevated above the level of the heart to reduce swelling and improve blood flow.
4. Cold compress: Applying a cold compress to the affected area can help reduce pain and swelling.
5. Monitoring: Monitor the person's vital signs, including their temperature, pulse, and breathing rate, to ensure there are no complications.
6. Wound care: If there is an open wound, clean and dress it regularly to prevent infection.
7. Rehydration: Encourage the person to drink plenty of fluids to stay h

[({'temperature': 0.3,
   'top_p': 0.95,
   'max_tokens': 512,
   'top_k': 50,
   'repeat_penalty': 1.1},
  "If someone has fractured their leg while hiking, it is essential to seek medical attention as soon as possible. The following precautions and treatment steps can be taken:\n\n1. Stop any bleeding: Apply direct pressure to the wound to stop any bleeding. Elevate the affected limb above heart level to reduce swelling.\n2. Immobilize the leg: Use a splint or brace to immobilize the leg and prevent further movement. This will help reduce pain and prevent further damage.\n3. Assess for neurovascular injury: Check for numbness, tingling, or loss of sensation in the affected limb. Also, assess for pulse and temperature changes.\n4. Manage pain: Administer pain medication as prescribed by a healthcare professional.\n5. Monitor for infection: Keep the wound clean and dry, and monitor for signs of infection such as redness, swelling, or increased pain.\n6. Consider surgery: Depending on t

#### Observation

The treatment of physical brain tissue injury depends on its severity and location. Core interventions include medications, surgery, and rehabilitation therapies (physical, occupational, speech). Some responses also mention lifestyle changes, emotional support, assistive devices, and advanced methods like stem cell or brain stimulation therapies. Personalized care from a multidisciplinary team is emphasized across all responses.

## Data Preparation for RAG

### Loading the Data

In [19]:
import json
import tiktoken

#import pandas as pd

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.vectorstores import Chroma

from google.colab import userdata, drive

In [20]:
# uncomment and run the below code snippets if the dataset is present in the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
pdf_path = r"/content/drive/MyDrive/AI project/Project_ Natural Language Processing with Generative AI_ Medical Assistant/medical_diagnosis_manual.pdf"

In [22]:
!pip install pypdf

In [23]:
pdf_loader = PyPDFLoader(pdf_path)

In [24]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

### Data Overview

#### Checking the first 5 pages

In [14]:
# Load all pages
all_pages = pdf_loader.load()

# View the first 5 pages
first_five_pages = all_pages[:5]

# Print or inspect
for i, page in enumerate(first_five_pages):
    print(f"\n--- Page {i+1} ---")
    print(page.page_content[:1000])  # Show first 1000 characters for readability


--- Page 1 ---
ankush.waghmare3804@gmail.com
VW1ASY76X3
This file is meant for personal use by ankush.waghmare3804@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

--- Page 2 ---
ankush.waghmare3804@gmail.com
VW1ASY76X3
This file is meant for personal use by ankush.waghmare3804@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

--- Page 3 ---
Table of Contents
1
Front  
  ................................................................................................................................................................................................................
1
Cover  
  .......................................................................................................................................................................................................
2
Front Matter  
  ............................................................................................

#### Checking the number of pages

In [15]:
# Check total number of pages
print("Total number of pages:", len(all_pages))

Total number of pages: 4114


### Data Chunking

In [25]:
document_chunks = pdf_loader.load_and_split(text_splitter)

### Embedding

In [5]:
!pip uninstall numpy -y
!pip install numpy==1.24.4
!pip install sentence-transformers
!pip install -U langchain-huggingface

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.27 requires numpy>=1.26.2; python_version < "3.13", but you have numpy 1.24.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
pymc 5.25.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tsfresh 0.

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

#### Observation
Now that we have chunked the raw input, we can present these chunks to an embedding model and then store the generated embeddings into a vector database.

### Vector Database

In [12]:
report = 'Medical_KnowledgeBase'
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    collection_name=report
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 6}
)

#### Observation

Initializes a vector database (**also called vector store**) using Chroma, a popular open-source vector database, to store document embeddings for retrieval in a Retrieval-Augmented Generation (**RAG**) system.

### Retriever

#### Observation
Initializes a retriever from the Chroma vector store to fetch similar documents based on embeddings.

### System and User Prompt Template

In [36]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama    # This will throw an error if you've have installed the GPU version of llama_cpp and haven't enabled the GPU

In [37]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [38]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

In [39]:
lcpp_llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # Context window
    n_batch=1024,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [40]:
qna_system_message = """
You are an assistant whose work is to give answers to questions with repect to a context.
User input will have the context required by you to answer user questions.

This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Strictly answer only using the information provided in the ###Context.
Do not mention anything about the information in ###Context or the question in ###Question in your final answer.

If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".

Remember that the answer to ###Question might not always be directly present in the information provided in the ###Context.
the answer can be indirectly derived from the information in ###Context.

"""

In [41]:
qna_user_message_template = """Answer the following question using only the context provided.
If the answer is not present in the context, say "The answer is not available in the document."

### Question:
{question}

### Context:
{context}
"""

- system message: This is the instruction that has to be given to the LLM.
- user message template: This is a message template that contains the context retrieved from the document chunks and the User Query.

### Response Function

In [42]:
def generate_rag_response(user_input , llm):
    """
    Args:
        user_input: User query for retrieving context and generating answer.
        llm: LLM to be used for generating the answer.
    Returns:
        Final answer string.
    """
    # Get relevant docs
    relevant_document_chunks = retriever.get_relevant_documents(user_input)
    context_list = [d.page_content.replace("\t", " ") for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Format combined prompt using the context and question
    full_prompt = f"""[INST]
{qna_system_message}

{qna_user_message_template.format(context=context_for_query, question=user_input)}
[/INST]"""

    try:
        response = llm(
            prompt=full_prompt,
            max_tokens=256,
            temperature=0,
            top_p=0.95,
            repeat_penalty=1.2,
            top_k=50,
            stop=['[/INST]'],
            echo=False
        )

        prediction = response["choices"][0]["text"].strip()

    except Exception as e:
        prediction = f'Sorry, I encountered the following error: \n {e}'

    return prediction

### Testing the RAG based system

In [ ]:
print(generate_rag_response("What is hypertension?" , lcpp_llm))

Llama.generate: prefix-match hit


Based on the context provided, I can answer your question as follows:

What is hypertension?

Hypertension is sustained elevation of resting systolic blood pressure (≥ 140 mm Hg), diastolic blood pressure (≥ 90 mm Hg), or both. It is a common condition that can lead to severe cardiovascular, neurologic, renal, and retinal complications if left untreated. The higher the blood pressure and the more severe the retinal changes and other evidence of target-organ involvement, the worse is the prognosis. Hypertension has no cure, but control of blood pressure can significantly limit adverse consequences.


#### Observation
We can see that it can generate meaningful answers within the context that has been provided.

In [ ]:
print(generate_rag_response("What are LLMs? What are the types of LLMs?" , lcpp_llm))

Llama.generate: prefix-match hit


Sure! I'm ready to answer your questions based on the provided context. Please go ahead and ask your first question. Remember, I can only provide answers based on the information provided in the context.


#### Observation
It is not able to generate the answer because the query is not relevant to the context.

## Question Answering using RAG

In [43]:
#Generates and compares multiple LLM responses to a medical question using different prompt tuning parameters. .
#Useful for evaluating how decoding settings impact answer quality.

def generate_rag_responses_with_tuning_qna(user_question, llm, retriever):
    """
    Generate multiple RAG responses using different LLM parameters,
    based on provided system and user message templates.

    Args:
        user_question (str): The user’s medical query.
        llm: The LLM instance to query.
        retriever: A retriever for fetching relevant documents.

    Returns:
        List of dictionaries, each with the parameter set and response text.
    """

    # --- Templates ---
    qna_system_message = """
You are an assistant whose work is to give answers to questions with repect to a context.
User input will have the context required by you to answer user questions.

This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Strictly answer only using the information provided in the ###Context.
Do not mention anything about the information in ###Context or the question in ###Question in your final answer.

If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".

Remember that the answer to ###Question might not always be directly present in the information provided in the ###Context.
the answer can be indirectly derived from the information in ###Context.
"""

    qna_user_message_template = """Answer the following question using only the context provided.
If the answer is not present in the context, say "The answer is not available in the document."

### Question:
{question}

### Context:
{context}
"""

    # Retrieve relevant document chunks
    retrieved_docs = retriever.get_relevant_documents(user_question)
    context_chunks = [doc.page_content.replace("\t", " ") for doc in retrieved_docs]
    context = "\n\n".join(context_chunks[:5])  # Use top 5 chunks

    # LLM parameter combinations
    param_combos = [
        {"temperature": 0.3, "top_p": 0.95, "max_tokens": 512, "top_k": 50, "repeat_penalty": 1.1},
        {"temperature": 0.7, "top_p": 0.9,  "max_tokens": 512, "top_k": 40, "repeat_penalty": 1.0},
        {"temperature": 0.1, "top_p": 1.0,  "max_tokens": 1024, "top_k": 100, "repeat_penalty": 1.2},
        {"temperature": 0.5, "top_p": 0.85, "max_tokens": 768, "top_k": 60, "repeat_penalty": 1.15},
        {"temperature": 0.9, "top_p": 0.95, "max_tokens": 512, "top_k": 80, "repeat_penalty": 1.0},
    ]

    all_responses = []

    # Loop through each parameter combo
    for i, params in enumerate(param_combos):
        prompt = f"""[INST]
{qna_system_message}

{qna_user_message_template.format(context=context, question=user_question)}
[/INST]"""

        print(f"\n=== Response {i+1} === [Params: {params}]")

        try:
            response = llm(
                prompt=prompt,
                temperature=params["temperature"],
                top_p=params["top_p"],
                max_tokens=params["max_tokens"],
                top_k=params["top_k"],
                repeat_penalty=params["repeat_penalty"],
                echo=False,
                stop=["[/INST]"]
            )

            output_text = response["choices"][0]["text"].strip()

        except Exception as e:
            output_text = f"Error: {str(e)}"

        all_responses.append({
            "params": params,
            "response": output_text
        })

        print(output_text)

    return all_responses


### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(generate_rag_responses_with_tuning_qna("What is the protocol for managing sepsis in a critical care unit?" , lcpp_llm, retriever))


=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


Based on the provided context, I cannot answer the question "What is the protocol for managing sepsis in a critical care unit?" as it is not directly mentioned in the given text. However, I can provide some information about sepsis and its management based on the context.

Sepsis is an inflammatory response to infection, which can lead to organ dysfunction and death. The management of sepsis involves early recognition, prompt administration of antibiotics, and supportive care to address any organ dysfunction.

The text mentions that vancomycin should be added if resistant staphylococci or enterococci are suspected, and that the patient's condition will continue to deteriorate despite antibiotic therapy unless septic foci are eliminated. Additionally, the text suggests that corticosteroid therapy may be beneficial for severe sepsis and septic shock.

However, the specific protocol for managing sepsis in a critical care unit is not described in the provided context. It is important to no

Llama.generate: prefix-match hit


Based on the provided context, I cannot answer the question "What is the protocol for managing sepsis in a critical care unit?" as the information provided does not directly address this question. However, based on the information provided, I can suggest that the protocol for managing sepsis in a critical care unit may include the following:

1. Early recognition and aggressive fluid resuscitation to maintain vital organ perfusion.
2. Empiric broad-spectrum antibiotics targeting likely pathogens, with prompt modification based on culture and sensitivity results.
3. Management of underlying infections, such as abscess drainage and surgical excision of infected tissues.
4. Glucose control with insulin therapy to maintain normal blood glucose levels.
5. Supportive care, including mechanical ventilation, dialysis, and cardiovascular support as needed.
6. Use of corticosteroids and activated protein C in severe sepsis and septic shock, with careful consideration of potential risks and benef

...

Based on the provided context, I cannot answer your question as it does not contain any information about managing sepsis in a critical care unit. The context provides general information about critically ill patients and the importance of early detection and treatment of sepsis, but it does not provide specific guidelines or protocols for managing sepsis in a critical care unit. Therefore, I cannot answer your question based on the provided context.

=== Response 4 === [Params: {'temperature': 0.5, 'top_p': 0.85, 'max_tokens': 768, 'top_k': 60, 'repeat_penalty': 1.15}]


Llama.generate: prefix-match hit


Based on the context provided, I cannot answer the question "What is the protocol for managing sepsis in a critical care unit?" as it is not directly mentioned in the text. However, I can provide information on the general approach to managing sepsis based on the content of the chapter.

The management of sepsis involves early recognition and aggressive fluid resuscitation, antibiotic therapy, and supportive care. The protocol for managing sepsis in a critical care unit should include prompt administration of broad-spectrum antibiotics, source control of the infection, and close monitoring of vital signs and laboratory values. Additionally, the use of vasopressors to maintain mean arterial pressure ≥ 65 mmHg and the management of hypotension with fluid resuscitation are important components of sepsis protocols.

It is essential to note that the specific protocol for managing sepsis may vary depending on the institution and the severity of the illness. Therefore, it is crucial to consul

Llama.generate: prefix-match hit


Based on the provided context, I cannot answer the question "What is the protocol for managing sepsis in a critical care unit?" as it is not directly stated in the provided text. However, I can provide information on the general approach to managing sepsis based on the context.

The approach to managing sepsis in a critical care unit involves a multidisciplinary team and a systematic approach to evaluate and treat the underlying infection and organ dysfunction. The management of sepsis includes prompt administration of antibiotics, source control, and supportive care to address any underlying organ dysfunction.

The specific protocol for managing sepsis may vary depending on the individual case and the clinical context, but it is generally based on the Surviving Sepsis Campaign guidelines, which emphasize the importance of early recognition, prompt antibiotic therapy, and aggressive supportive care.

In summary, while the specific protocol for managing sepsis in a critical care unit is

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [26]:
print(generate_rag_responses_with_tuning_qna("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" , lcpp_llm,retriever))

/tmp/ipython-input-21-45088643.py:48: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(user_question)



=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]
Sure! I'm ready to answer your questions based on the provided context. Please go ahead and ask your first question. Remember to preface your question with the token "#Question" to avoid any confusion.

=== Response 2 === [Params: {'temperature': 0.7, 'top_p': 0.9, 'max_tokens': 512, 'top_k': 40, 'repeat_penalty': 1.0}]


Llama.generate: prefix-match hit


###Context:

• Surgical removal
• IV fluids and antibiotics
• Treatment of acute appendicitis is open or laparoscopic appendectomy
• Perforated appendix should be removed
• Inflammatory bowel disease involving the cecum is a contraindication to appendectomy
• Antibiotics should be continued until the patient's temperature and WBC count have normalized or until a fixed course is completed

###Question:

What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Based on the context, the answer to the question is:

Common symptoms of appendicitis include epigastric or periumbilical pain, nausea, vomiting, and anorexia. However, these symptoms may not always be present or may be atypical. The only way to confirm the diagnosis is through surgical intervention. Appendectomy is the recommended treatment for appendicitis, and it is not curable through medicine alone. If the appendix is perforated, antibioti

Llama.generate: prefix-match hit


###Question: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

###Context:
• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate: In these cases, it usually lies behind the cecum or the ileum and mesentery of the right colon. A contraindication to appendectomy is inflammatory bowel disease involving the cecum. However, in cases of terminal ileitis and a normal cecum, the appendix should be removed.
Appendectomy should be preceded by IV antibiotics. Third-generation cephalosporins are preferred. For nonperforated appendicitis, no further antibiotics are required. If the appendix is perforated, antibiotics should b

Llama.generate: prefix-match hit


Based on the provided context, I can answer your question as follows:

### Question: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Answer: The common symptoms of appendicitis include epigastric or periumbilical pain, brief nausea and vomiting, anorexia, and a shift in pain to the right lower quadrant after a few hours. However, these classic findings may not be present in all cases. Treatment for appendicitis is typically surgical removal of the inflamed appendix, with antibiotics being used supportively to cover any secondary infections. If the appendix is perforated, antibiotics should be continued until the patient's temperature and WBC count have normalized or until a fixed course has been completed. Surgical procedure for appendicitis includes open or laparoscopic appendectomy. In cases where a large inflammatory mass is found involving the appendix, terminal ileum, and cecum, resec

Llama.generate: prefix-match hit


Based on the provided context, the answer to the question "What are the common symptoms for appendicitis, and can it be cured via medicine?" is:

Common symptoms for appendicitis include epigastric or periumbilical pain, brief nausea, vomiting, and anorexia, followed by pain shift to the right lower quadrant, tenderness in the right lower quadrant, and low-grade fever. However, these classic findings may not be present in all patients.

Appendicitis can be treated with medicine, specifically antibiotics, but prompt surgical removal of the inflamed appendix is necessary to prevent complications and ensure a successful outcome. Surgery is the only definitive treatment for appendicitis, and delaying it increases the risk of perforation and subsequent complications.

Please note that I have only answered the question based on the provided context and have not referred to any external information.
[{'params': {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalt

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [27]:
print(generate_rag_responses_with_tuning_qna("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" , lcpp_llm,retriever))


=== Response 1 === [Params: {'temperature': 0.3, 'top_p': 0.95, 'max_tokens': 512, 'top_k': 50, 'repeat_penalty': 1.1}]


Llama.generate: prefix-match hit


Based on the provided context, I can answer your question. The possible causes of sudden patchy hair loss, such as localized bald spots on the scalp, include:

1. Androgenetic alopecia (male-pattern and female-pattern hair loss)
2. Alopecia areata
3. Traction alopecia
4. Tinea capitis
5. Trichotillomania
6. Scarring alopecia (such as central centrifugal scarring alopecia, dissecting cellulitis of the scalp, and acne keloidalis nuchae)
7. Lichen planopilaris
8. Chronic cutaneous lupus lesions
9. Tinea versicolor
10. Psoriasis

Please note that this list is not exhaustive and other causes of hair loss may exist. Additionally, some of these conditions may have overlapping symptoms or co-occur with one another. A thorough evaluation by a healthcare professional is necessary for an accurate diagnosis and appropriate treatment.

=== Response 2 === [Params: {'temperature': 0.7, 'top_p': 0.9, 'max_tokens': 512, 'top_k': 40, 'repeat_penalty': 1.0}]


Llama.generate: prefix-match hit


###Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

###Context:
corticosteroids, retinoids, or immunosuppressants.
Hair loss due to chemotherapy is temporary and is best treated with a wig; when hair regrows, it may be
different in color and texture from the original hair. Hair loss due to telogen effluvium or anagen effluvium
is usually temporary as well and abates after the precipitating agent is eliminated. Key Points
• Androgenetic alopecia (male-pattern and female-pattern hair loss) is the most common type of hair loss.
• Concomitant virilization in women or scarring hair loss should prompt a thorough evaluation for the
underlying disorder.
• Microscopic hair examination or scalp biopsy may be required for definitive diagnosis.
Alopecia Areata
Alopecia areata is sudden patchy hair loss in people with no obvious skin or systemic disor

...

Based on the provided context, I can answer your question. The possible causes behind sudden patchy hair loss with localized bald spots on the scalp are:

1. Alopecia Areata: This is an autoimmune disorder that causes hair loss in patches. It is thought to be triggered by unknown environmental factors and is more common in people with a family history of the condition.
2. Androgenetic alopecia (male-pattern and female-pattern hair loss): This is the most common cause of hair loss, caused by hormonal imbalances that lead to gradual shrinking of hair follicles.
3. Traction alopecia: This occurs due to physical stress or tension on the scalp, such as from tight hairstyles like braids or ponytails.
4. Tinea capitis (ringworm infection): This is a fungal infection that can cause hair loss and scaling of the skin.
5. Trichotillomania: This is a psychological disorder characterized by compulsive hair pulling, leading to hair loss.
6. Scarring alopecia: This occurs due to inflammation or damag

Llama.generate: prefix-match hit


Based on the provided context, there are several possible causes for sudden patchy hair loss, including:

1. Androgenetic alopecia: This is the most common cause of hair loss in both men and women, and it is characterized by a gradual thinning of the hair on the top of the head and a receding hairline in men.
2. Alopecia areata: This is an autoimmune disorder that can cause sudden patchy hair loss on the scalp or other parts of the body.
3. Traction alopecia: This is caused by physical traction or stress to the scalp, such as from tight hairstyles or hair extensions.
4. Tinea capitis: This is a fungal infection of the scalp that can cause patchy hair loss and scaling of the skin.
5. Trichotillomania: This is a psychological disorder characterized by the compulsive pulling out of one's own hair, leading to sudden patchy hair loss.

Based on the provided context, it seems that the most likely cause for the patient's symptoms is alopecia areata, as they have experienced sudden patchy hair

Llama.generate: prefix-match hit


Based on the information provided in the context, there are several possible causes of sudden patchy hair loss, including:

1. Androgenetic alopecia: This is the most common cause of hair loss, and it is characterized by the gradual thinning of hair on the top of the head.
2. Alopecia areata: This is a condition where hair loss occurs in patches, and it is often associated with an autoimmune disorder.
3. Traction alopecia: This is a type of hair loss that is caused by the constant pulling or tugging on the hair, such as with tight hairstyles or hair extensions.
4. Telogen effluvium: This is a condition where there is a significant increase in the number of hair follicles that enter the resting phase, leading to excessive hair shedding.
5. Anagen effluvium: This is a condition where there is a disruption of the growing phase of the hair follicle, leading to abnormal loss of anagen hairs.

Based on the information provided, the treatment for sudden patchy hair loss will depend on the und

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(generate_rag_responses_with_tuning_qna("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" , lcpp_llm,retriever))

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
print(generate_rag_responses_with_tuning_qna("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" , lcpp_llm,retriever))

## Output Evaluation

In [22]:
groundedness_rater_system_message  = """

You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

Please note: Make sure you give a single overall rating in the range of 1 to 5 along with an overall explanation.

"""

In [23]:
relevance_rater_system_message =  """

You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

Note: Provide a single overall rating in the range of 1 to 5, along with a brief explanation of why you assigned that score.
"""

In [24]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [25]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

## Actionable Insights and Business Recommendations

* The RAG-based output is superior to the generic LLM output because it provides a more context-aware and strategically relevant response. While the generic output lists common applications of LLMs in the legal domain, it lacks specificity and mainly presents a laundry list of use cases without much depth or insight.

* In contrast, the RAG output offers a more tailored and innovative perspective, discussing unique scenarios such as Private LLMs, Open Source LLMs, and Sovereign LLMs, with relevant examples like BloombergGPT and SoftBank's localized Japanese LLM.

* This makes the RAG output more credible and informative, as it connects specific examples to broader industry trends and motivations, such as the need for customizability, data sovereignty, and strategic localization.

* Additionally, the RAG output is more coherent and contextually connected, organizing information into meaningful categories that enhance the clarity and narrative flow. This results in a more engaging and valuable answer, providing deeper insights into the evolving landscape of LLM applications compared to the more informational and generic approach of the LLM output.

* Using a Retrieval-Augmented Generation (RAG) system with medical data enables the generation of meaningful insights and recommendations that can guide healthcare decisions and operational strategies. For instance, when querying about sepsis protocols in critical care, the RAG system may surface key actions such as early antibiotic administration, fluid resuscitation, and lactate monitoring. From these responses, actionable insights can be derived—such as identifying inconsistencies in how quickly antibiotics are administered across departments or missing lactate checks during triage. These insights support business recommendations like standardizing training on the “golden hour” response, integrating clinical checklists in EHR systems, or enriching the knowledge base with updated dosage guidelines. More broadly, RAG systems can reveal patterns, gaps, or ambiguities in clinical workflows, enabling decisions like introducing AI-based alerts, harmonizing protocols, or investing in clinician education. Ultimately, these insights improve patient care, reduce medical errors, and enhance compliance with clinical standards.

<font size=6 color='blue'>Power Ahead</font>
___